In [1]:
import classes
import random
from amadeus import Client, ResponseError
from hotels1 import hotelFunctions
import amadeus
#import api
import hotels1
import hotelTests
import transport
from transport import transportFunctions
from transport import getNZfromCity
import datetime

In [2]:
amadeus = Client(
        client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
        client_secret='rKvILHDsjxcCh6yq'
    )
hotel = hotelFunctions(amadeus)

In [4]:
hotels = hotel.getHotelsByGeocode(getNZfromCity('London')[0], getNZfromCity('London')[1])

Those are it: 'media'


In [5]:
for i in hotels:
    print(i)

{'Duration': None, 'Cost': 219.0, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'HOLIDAY INN EXP SOUTHWARK ST', 'Dest': '51.5061,-0.10144'}
{'Duration': None, 'Cost': 196.63, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'London Blackfriars', 'Dest': '51.5131,-0.10561'}
{'Duration': None, 'Cost': 207.63, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'London Waterloo', 'Dest': '51.50329,-0.11521'}
{'Duration': None, 'Cost': 367.0, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'Andaz Liverpool Street London', 'Dest': '51.51651,-0.08191'}
{'Duration': None, 'Cost': 449.0, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'THE WALDORF HILTON', 'Dest': '51.51264,-0.11937'}
{'Duration': None, 'Cost': 285.0, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'DOUBLETREE TOWER OF LONDON', 'Dest': '51.51091,-0.07849'}
{'Duration': None, 'Cost': 202.63, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'London Tower Bridge', 'Dest': '51.50003,-0.0815'}
{'Duration': None, 'Cost': 221.63, 

In [3]:
transport = transportFunctions()
transports = transport.getTransport(51.5061,-0.10144,51.2191,-0.1246)


In [7]:
for i in transports:
    print(i)

{'Duration': datetime.timedelta(seconds=360), 'Cost': 0, 'Times:': [datetime.datetime(2022, 6, 18, 17, 2), datetime.datetime(2022, 6, 18, 17, 8)], 'Title': 'pedestrian', 'Dest': '51.508517,-0.103367'}
{'Duration': datetime.timedelta(seconds=360), 'Cost': 0, 'Times:': [datetime.datetime(2022, 6, 18, 17, 8), datetime.datetime(2022, 6, 18, 17, 14)], 'Title': 'pedestrian', 'Dest': '51.51181,-0.10333'}
{'Duration': datetime.timedelta(seconds=2280), 'Cost': None, 'Times:': [datetime.datetime(2022, 6, 18, 17, 14), datetime.datetime(2022, 6, 18, 17, 52)], 'Title': 'regionalTrain', 'Dest': '51.2402,-0.1659'}
{'Duration': datetime.timedelta(seconds=240), 'Cost': None, 'Times:': [datetime.datetime(2022, 6, 18, 18, 0), datetime.datetime(2022, 6, 18, 18, 4)], 'Title': 'regionalTrain', 'Dest': '51.22681,-0.13253'}
{'Duration': datetime.timedelta(seconds=60), 'Cost': 0, 'Times:': [datetime.datetime(2022, 6, 18, 18, 4), datetime.datetime(2022, 6, 18, 18, 5)], 'Title': 'pedestrian', 'Dest': '51.226568,

In [8]:
attractions = hotel.getActivities(32.079664, 34.767410, 2)

In [9]:
for i in attractions:
    print(i)

{'Duration': None, 'Cost': 25.0, 'Times:': [None, None], 'Title': 'Tel Aviv Pub Crawl', 'Dest': '32.075996,34.766022'}
{'Duration': None, 'Cost': 96.0, 'Times:': [None, None], 'Title': 'From Jerusalem: Masada and Dead Sea Tour', 'Dest': '32.075565,34.767686'}
{'Duration': None, 'Cost': 34.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Street Food Tour', 'Dest': '32.077840,34.777979'}
{'Duration': None, 'Cost': 80.0, 'Times:': [None, None], 'Title': 'Jerusalem Day Tour from Tel Aviv', 'Dest': '32.073954,34.765815'}
{'Duration': None, 'Cost': 65.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Jerusalem Old City Late Morning Tour', 'Dest': '32.074602,34.768443'}
{'Duration': None, 'Cost': 53.0, 'Times:': [None, None], 'Title': 'Tel Aviv: Private Group Guided Bike Tour', 'Dest': '32.074815,34.767932'}


In [10]:
def chooseCheapestHotel(hotels):
    lowest = 1000000
    index = 0
    for i in range(0, len(hotels)):
        if (hotels[i].cost < lowest):
            index = i
            lowest = hotels[i].cost
    return hotels[index]
def chooseMostLuxuriousHotel(hotels):
    highest = 0
    index = 0
    for i in range(0, len(hotels)):
        if (hotels[i].cost > highest):
            index = i
            highest = hotels[i].cost
    return hotels[index]

In [11]:
lowCost = False
luxury = True
#shoomerShabat = False
tripPeople = 4
duration = 4
museumOriented = True
fastPaced = True
usedAttractions = []
maxPrice = 10000
priceCapLeft = 10000 # maybe add mostCentralLocation implementation later
amadeus = Client(
    client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
    client_secret='rKvILHDsjxcCh6yq'
)
city = "New York"
hotel = hotelFunctions(amadeus)
transport = transportFunctions()
#print(city)
#print(getNZfromCity(city))
hotels = hotel.getHotelsByGeocode(getNZfromCity(city)[0], getNZfromCity(city)[1])
for i in hotels:
    print(i)
choosenHotel = None
if (lowCost):
    choosenHotel = chooseCheapestHotel(hotels)
elif (luxury):
    choosenHotel = chooseMostLuxuriousHotel(hotels)
else:
    choosenHotel = random.choice(hotels)
print(choosenHotel)

{'Duration': None, 'Cost': 319.06, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'Holiday Inn Brooklyn Downtown', 'Dest': '40.68739,-73.98265'}
{'Duration': None, 'Cost': 285.38, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'Sheraton Brooklyn New York Hotel', 'Dest': '40.6916,-73.98438'}
{'Duration': None, 'Cost': 362.67, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'New York Marriott at the Brooklyn Bridge', 'Dest': '40.69384,-73.98916'}
{'Duration': None, 'Cost': 313.33, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'Best Western Plus Arena Hotel', 'Dest': '40.67837,-73.94654'}
{'Duration': None, 'Cost': 266.28, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'Fairfield Inn by Marriott New York Manhattan Financial District', 'Dest': '40.70604,-74.00502'}
{'Duration': None, 'Cost': 381.03, 'Times:': ['2022-06-18', '2022-06-19'], 'Title': 'HAMPTON INN MANHATTAN SEAPORT', 'Dest': '40.70906,-74.00199'}
{'Duration': None, 'Cost': 478.58, 'Times:': ['2022-06-18', '2022-06-19

In [12]:
def cleanUsedAttractions(attractions, usedAttractions):
    newAttractionList = []
    for i in attractions:
        if (i.title.lower() in usedAttractions):
            continue
        newAttractionList.append(i)
    return newAttractionList

In [ ]:
def augDate(minute, hour, second, year, month, day)

In [16]:
transport = transportFunctions()
def planFullDay(tripPeople, luxury, lowCost, museumOriented, 
                fastPaced, priceCapLeft, attractionObject, transportObject, hotel, usedAttractions, date):
    xCoord = hotel.destination.split(',')
    yCoord = float(xCoord[1])
    xCoord = float(xCoord[0])
    print(xCoord, yCoord)
    attractions = [] # maybe add most_central_location implementation later
    while (attractions == []):
        attractions = attractionObject.getActivities(xCoord, yCoord, 10)
        attractions = cleanUsedAttractions(attractions, usedAttractions)
        noise = random.random() - 0.5 * 0.2
        xCoord += noise
        yCoord += noise
    attractionOptions = []
    for i in attractions:
        print(i)
    if (museumOriented):
        for i in attractions:
            if ('museum' in i.title.lower()):
                attractionOptions.append(i)
        if (attractionOptions == []):
            attractionOptions = attractions
            noise = random.random() - 0.5 * 0.2
            xCoord += noise
            yCoord += noise
    else:
        attractionOptions = attractions
    if (lowCost):
        attractionOptions.sort(key=lambda x: x.cost)
    elif (luxury):
        attractionOptions.sort(key=lambda x: x.cost, reverse=True)
    day, usedAttractions = planDayObject(fastPaced, attractionOptions, transportObject, xCoord, yCoord)

In [17]:
print(planFullDay(tripPeople, luxury, lowCost,
            museumOriented, fastPaced, priceCapLeft, hotel, transport, choosenHotel, usedAttractions))

40.70541 -74.00783
{'Duration': None, 'Cost': 26.0, 'Times:': [None, None], 'Title': '9-11 Memorial and Museum tickets', 'Dest': '40.711548,-74.013292'}
{'Duration': None, 'Cost': 195.0, 'Times:': [None, None], 'Title': 'Big Apple Helicopter Tour of New York', 'Dest': '40.712434,-74.010543'}
{'Duration': None, 'Cost': 36.0, 'Times:': [None, None], 'Title': 'NYC One World Observatory Skip-the-Line Ticket', 'Dest': '40.711544,-74.013487'}
{'Duration': None, 'Cost': 39.0, 'Times:': [None, None], 'Title': 'Empire State Building Tickets - Observatory and Optional Skip the Line Tickets', 'Dest': '40.712784,-74.005941'}
{'Duration': None, 'Cost': 42.0, 'Times:': [None, None], 'Title': 'Circle Line: Complete Manhattan Island Cruise', 'Dest': '40.712784,-74.005941'}
{'Duration': None, 'Cost': 39.0, 'Times:': [None, None], 'Title': 'Top of the Rock Observation Deck, New York', 'Dest': '40.712784,-74.005941'}
{'Duration': None, 'Cost': 27.0, 'Times:': [None, None], 'Title': '9/11 Memorial Museum 

In [11]:
transport = transportFunctions()
#print(datetime.datetime.now())
date = datetime.datetime.now()
date = date.replace(minute=59, hour=23, second=59, year=2022, month=10, day=1)
a = transport.getTransportByTime(51.5061,-0.10144,51.2191,-0.1246, str(date.isoformat()))
for i in a:
    print(i)
    pass

{'routes': [{'id': 'C0', 'sections': [{'id': 'C0-S0', 'type': 'pedestrian', 'departure': {'time': '2022-10-02T06:50:00+01:00', 'place': {'type': 'place', 'location': {'lat': 51.5061, 'lng': -0.10144}}}, 'arrival': {'time': '2022-10-02T07:08:00+01:00', 'place': {'name': 'London Bridge Rail Station', 'type': 'accessPoint', 'location': {'lat': 51.505327, 'lng': -0.086886}}}, 'transport': {'mode': 'pedestrian'}}, {'id': 'C0-S0-arr', 'type': 'pedestrian', 'departure': {'time': '2022-10-02T07:08:00+01:00', 'place': {'name': 'London Bridge Rail Station', 'type': 'accessPoint', 'location': {'lat': 51.505327, 'lng': -0.086886}}}, 'arrival': {'time': '2022-10-02T07:09:00+01:00', 'place': {'name': 'London Bridge', 'type': 'station', 'location': {'lat': 51.50511, 'lng': -0.08609}, 'wheelchairAccessible': 'yes', 'id': '120011305', 'code': 'LNDNBDC'}}, 'spans': [{'walkAttributes': ['indoor']}], 'transport': {'mode': 'pedestrian'}}, {'id': 'C0-S1', 'type': 'transit', 'departure': {'time': '2022-10-02

In [ ]:
def planFullDay(fastPaced, attractionOptions, transportObject, xCoord, yCoord, date, usedAttractions):
    currDate = date.replace(minute = 0, hour = 9, second=0)
    attractionArr = []
    transportArr = []
    currX = xCoord
    currY = yCoord
    attractionCount = 2
    if (fastPaced):
        attractionCount = 3
    for i in range(0, attractionCount):
        attractionLocation = attractionOptions[i].destination.split(',')
        destX = float(attractionLocation[0])
        destY = float(attractionLocation[1])
        currTransport = transport.getTransportByTime(currX, currY, destX, destY, str(currDate.isoformat()))
        print(transport)
        currDate = transport
        transportArr.append()
        attractionArr.append(attractionOptions[i])
        usedAttractions.append(attractionOptions[i].title.lower())